# Algebraic $E_\infty$-bialgebra on cubes

In this notebook we describe a natural $E_\infty$-bialgebra structure on $C_\bullet(\mathrm I^d;\mathbb F_2)$, the cellular chains of a standard cube, using the finitely presented $E_\infty$-prop of [[1]](https://arxiv.org/abs/1808.00854).

## The prop $\mathcal S$

Recall ...

## Chain on standard cubes
 Recall...

In [ ]:
def boundary(x):
    answer = set()
    for i, letter in enumerate(x):
        if letter == 'e':
            answer ^= {x[:i]+'0'+x[i+1:],
                       x[:i]+'1'+x[i+1:]}
    return answer

def dimension(x):
    return x.count('e')

## Join

We will model the function
$$\ast : C_\bullet(\mathrm I^d;\mathbb F_2) \otimes C_\bullet(\mathrm I^d;\mathbb F_2) \to C_{\bullet+1}(\mathrm I^d;\mathbb F_2)$$

associated to the degree one generator of $\mathcal S$.

We define

$$ \ast \big( (x_1,\dots x_d) \otimes (y_1,\dots y_d) \big) = 
\sum_{i=1}^d \big( y_1,\dots,y_{i-1}, (x_i\ast y_i), y_{i+1}, \dots, y_d \big).$$

where define $$\ast(x,y) = \begin{cases} 
e &  x=[0]\ y=[1] \text{ or} \\
  &  x=[1]\ y=[0] \\
0 & \text{otherwise}
\end{cases}$$

In [ ]:
def join(x,y):
    assert len(x)== len(y), 'not in the same cube'
    
    ast = {'01':'e','10':'e'}
    answer = set()
    for i, pair in enumerate(zip(x,y)):
        try:
            answer ^= {y[:i]
                      +ast[pair[0]+pair[1]]
                      +x[i+1:]}
        except KeyError:
            pass
    
    expected_dim = dimension(x)+dimension(y)+1
    answer = {term for term in answer if 
              term.count('e') == expected_dim}
    
    return answer

def display_join(x,y):
    jn = join(x,y) 
    if jn:
        return f'[{x}]*[{y}] = [' + ']+['.join(jn) + ']'
    if not jn:
        return f'[{x}]*[{y}] = 0'

### Example
Given a choice of integer $d$, we print the join $x*y$ of basis elements in $C_\bullet(\mathrm I^d; \mathbb F_2)$ for which it is nonzero.

In [ ]:
from itertools import product

d = 1
elements = [''.join(i) for i in product(['0','1','e'], repeat = d)]

for x,y in product(elements, repeat=2):
    if join(x,y):
        print(display_join(x,y))

### Check attaching map
We define function that verifies that for any pair of basis elements $x,y \in C_\bullet(\mathrm I^d; \mathbb F_2)$

$$\partial (x \ast y) + (\partial x) \ast y + x \ast \partial y 
= \varepsilon(x)y + \varepsilon(y)x.$$


In [ ]:
def check_attaching(x,y):
#     print('x',x,'y',y)
    answer = set()
    for x_term in boundary(x):
        answer ^= join(x_term, y)
#     print('dx*y',answer)    
    for y_term in boundary(y):
        answer ^= join(x, y_term)
#     print('x*dy',answer)
    for z in join(x,y):
        answer ^= boundary(z)
#     print('d(x*y)',answer)
    if dimension(x) == 0:
        answer ^= {y}
#     print('c(x)y',answer)
    if dimension(y) == 0:
        answer ^= {x}
#     print('c(y)x',answer)
    return not bool(answer)